In [1]:
# Requests for handling HTTP get and other requests
import requests
import time # import for playing nice and not getting blocked
import pandas as pd
# from BeautifulSoup4 import BeatifulSoup if installed through pip install BeautifulSoup4
# 
from bs4 import BeautifulSoup

In [2]:
url = "https://www.ss.com/lv/real-estate/flats/riga/centre/sell/"
url

'https://www.ss.com/lv/real-estate/flats/riga/centre/sell/'

In [5]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup):
    column_list = ["description","url"] # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns
    return column_list

# lets make a function from the above doodle and make it work on most pages on SS
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

def getRow(row,colist):
    row_tds = row.find_all('td')
    rowDict = {}
    if len(row_tds) < 3: # a little sanity check
        print("Hmm bad row")
        return rowDict
    
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    for td,key in zip(row_tds[3:],colist[2:]): 
        rowDict[key] = td.text
    return rowDict

# so if we know how to work on single row then we can do process multiple rows
def getRows(rowlist,colist):
    return [getRow(row, colist=colist) for row in rowlist] # so return a list of dictionaries

# so with this function I can get full dataframe from a single page on ss.com not only apartments
def getDFfromURL(url):
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200:
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup)
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    return pd.DataFrame(rows, columns=column_names)

def getAllLocalUrls(url):
    """Get a list of all urls including paginated pages"""
    results = [url] # default is just the url if no extra pages found, teiksim Bolderājai...
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    # we just need a one element
    prevanchor = soup.find('a', {"rel":"prev"}) # find finds first match only
    if prevanchor == None: # means there is only one page of ads
        return results
    href = prevanchor.attrs.get('href')
    lastPageNum = int(href.split('/page')[-1].split('.html')[0])
    print("Last page is",lastPageNum)
    nurls = [f"{url}page{n}.html" for n in range(2,lastPageNum+1)]
    results += nurls
    return results

def get_all_ads_df(start_url, save_excel_path=None):
    df_list=[] # so we will save our dataframes in a list
    local_urls = getAllLocalUrls(start_url)
    for url in local_urls:
        print(f"Gathering data from {url}")
        df_list.append(getDFfromURL(url))
        time.sleep(0.3) # we need this to play nice! to avoid rate limit or IP ban!!
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
    big_df = pd.concat(df_list) # then make a big dataframe from all the small dataframes
    if save_excel_path:
        big_df.to_excel(save_excel_path)
    return big_df

In [6]:
volvo_url = "https://www.ss.com/lv/transport/cars/volvo/sell/"

In [7]:
df = get_all_ads_df(volvo_url, save_excel_path="volvo_270921.xlsx")

Last page is 45
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page2.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page3.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page4.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page5.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page6.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page7.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page8.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page9.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page10.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page11.html
Gathering data from https://www.ss.com/lv/transport/cars/volvo/sell/page12.html
Gathering data from https://www.ss.com/lv/t

In [8]:
df.shape

(1348, 7)

In [9]:
df.head()

,description,url,Modelis,Gads,Tilp.,Nobrauk.,Cena
0,Auto arī dzīvē tāds kā bildēs. Virsbūve ļoti l...,https://ss.com/msg/lv/transport/cars/volvo/v70...,V70,2002,2.4D,330 tūkst.,"2,300 €"
1,"Auto labā tehniskā un vizuālā stāvoklī, auto i...",https://ss.com/msg/lv/transport/cars/volvo/v70...,V70,2003,2.4D,390 tūkst.,"2,600 €"
2,Volvo XC 90 2.4 D5 Edition 147 KW / Webasto / ...,https://ss.com/msg/lv/transport/cars/volvo/xc9...,XC 90,2011,2.0D,235 tūkst.,"12,490 €"
3,Tikko No Vacijas. Facelift Modelis. Mehaniska ...,https://ss.com/msg/lv/transport/cars/volvo/xc9...,XC 90,2010,2.4D,-,"11,650 €"
4,"Volvo V70 Facelift, viens no labākajiem un jau...",https://ss.com/msg/lv/transport/cars/volvo/v70...,V70,2007,2.4D,300 tūkst.,"3,500 €"
